<a href="https://colab.research.google.com/github/naoya5614/Kaggle/blob/main/Future_Prediction_Of_Cloud_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 雲画像の未来予測

# [1] データの確認・把握

## 1.衛星画像データの確認・把握

### 1-1.衛星画像データのファイルパス

**ファイルパスとは、ファイルの住所を表す文字列を意味し、以下の2つに大別される**
```
*   絶対パス: 最上位の階層から見た目的のファイルの場所
*   相対パス: プログラムを実行する位置から見た目的のファイルの場所
```

### 1-2.衛星画像を読み込む

**画像の読み込むライブラリのインポート**
```
# OpenCVライブラリのインポート
import cv2
```
**画像を読み込む**
```
# cv2.imread()関数を使って、白黒画像を読み込む
image = cv2.imread(画像のファイルパス, 0)
```

In [ ]:
# ライブラリのインポート
import numpy as np
import cv2

# 2016年１月１日16時時点の衛星画像を指定するファイルパス
file_path = 'train/sat/2016-01-01/2016-01-01-16-00.fv.png'

# 衛星画像を読み込む
image = cv2.imread(file_path, 0)

# 出力
print(image)

### 1-3.衛星画像を構成する値の統計量を確認する

**読み込んだ衛星画像がどのような値で構成されているのかを確認する**
```
*   最大値　np.max()関数
*   最小値　np.min()関数
*   平均値　np.mean()関数
```
```
# NumPyライブラリのインポート
import numpy as np

# 最大値の取得
np.max(配列)

# 最小値の取得
np.min(配列)

# 平均値の取得
np.mean(配列)
```

In [ ]:
# ライブラリのインポート
import numpy as np
import cv2

# 2016年１月１日16時時点の衛星画像を指定するファイルパス
file_path = 'train/sat/2016-01-01/2016-01-01-16-00.fv.png'

# 衛星画像を読み込む
image = cv2.imread(file_path, 0)

# 最大値の出力
print(np.max(image))

# 最小値の出力
print(np.min(image))

# 平均値の出力
print(np.mean(image))

### 1-4.衛星画像を可視化する

**今回扱う衛星画像**
```
*   最小値0
*   最大値255
# 8bit(256階調)
```
**可視化ライブラリをインポートする**
```
# matplotlib.pyplotモジュールのインポート
import matplotlib.pyplot as plt
```
**画像を可視化する**
```
# 白黒画像の描画
plt.imshow(画像を表す配列, 'gray')
```

In [ ]:
# ライブラリのインポート
import numpy as np
import cv2
import matplotlib.pyplot as plt

# 2016年１月１日16時時点の衛星画像を指定するファイルパス
file_path = 'train/sat/2016-01-01/2016-01-01-16-00.fv.png'

# 衛星画像を読み込む
image = cv2.imread(file_path, 0)

# 衛星画像を描画する
plt.imshow(image, 'gray')

# グラフの出力
plt.show()

### 1-5.ファイルパスを一般化する① f文字列による文字列の置換

**規則性を捉えるために、文字列の構成成分を理解する**
```
# 下記のように一般化する
{trainもしくはtest}/sat/{年}-{月}-{日}/{年}-{月}-{日}-{時}-00.fv.png

衛星画像のファイルパスは
*   train もしくは test
*   年
*   月
*   日
*   時
の5つの成分から構成されています。


これらの成分に対応する５つの変数である
*   phase:train もしくはtest
*   year: 年
*   month: 月
*   day: 日
*   hour: 時間
を与える。

{phase}/sat/{year}-{month}-{day}/{year}-{month}-{day}-{hour}-00.fv.png
```

**f文字列と置換フィールド**
```
phase = 'train'
year = 2016
month = 1
day = 1
hour = 16

# f文字列による文字列の置換
file_path = f'{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png'

## 置換フィールド{}内の文字列が変数によって置換されることで、
## file_pathに代入される文字列は、'train/sat/2016-01-01/2016-01-01-16-00.fv.png'となる。
```

In [ ]:
# 衛星画像のファイルパスを特定するための５つの変数に値を代入する
phase = 'train'
year = 2016
month = 1
day = 1
hour = 16

# 「f文字列」による置換が可能な文字列をfile_pathに代入する
file_path = f'{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png'

print(file_path)

### 1-6.ファイルパスを一般化する② 日時の扱い方

**衛星画像の各ファイルパスを分ける差分となる変数について改めて確認する**
```
*   phase:train もしくはtest
*   year: 年
*   month: 月
*   day: 日
*   hour: 時間
# year、month、day、hourの４つは全て日時を表すもの
# phaseに関しても日時の情報によって文字列を特定できる
```
**datetimeとtimedelta**
```
datetime: 日時情報を表す
timedelta: 日時どうしの差分(経過時間)を表す
```
**datetime、timedeltaのインポート**
```
# datetime.datetimeクラスのインポート
from datetime import datetime as dt

# datetime.timedeltaクラスのインポート
from datetime import timedelta
```

**1. datetimeによる日時の操作**

```
# オブジェクト生成の際の引数に(年, 月, 日, 時, 分, 秒)をそれぞれ指定することで、日時を表すオブジェクトを作成できる

# 「2016年1月1日16時0分0秒」
date = dt(2016, 1, 1, 16, 0, 0)
```



**年, 月, 日, 時, 分, 秒の情報を、それぞれ単独で数値データとして抽出する**

```
# 「2016年1月1日16時0分0秒」
date = dt(2016, 1, 1, 16, 0, 0)


# dateから、「年」のみを数値データとして抽出する
## yearに2016が代入される
year = date.year

# dateから、「月」のみを数値データとして抽出する
## monthに1が代入される
month = date.month

# dateから、「日」のみを数値データとして抽出する
## dayに1が代入される
day = date.day

# dateから、「時」のみを数値データとして抽出する
## hourに16が代入される
hour = date.hour
```



**2. timedeltaによる日時の加算、減算**

```
# 「年」「日」「時」「分」「秒」の各単位ごとに、日時の差分を指定できる

単位	引数名
年	years
日	days
時	hours
分	minutes
秒	seconds

# 例) 2時間分の差分を表すtimedeltaオブジェクトを生成する
two_hours = timedelta(hours=2)
```

**datetimeオブジェクトの日時情報に対して変更を加える**
```
# 「2016年1月1日16時0分0秒」のdatetimeオブジェクト
date = dt(2016, 1, 1, 16, 0, 0)

# 2時間後を表すdatetimeオブジェクトを生成
date_after2hours = date + timedelta(hours=2)

# 3年前を表すdatetimeオブジェクトを生成
date_before3years = date - timedelta(years=3)
```

In [ ]:
# ライブラリのインポート
from datetime import datetime as dt
from datetime import timedelta

# 「2016年1月1日16時0分0秒」の日時オブジェクトを作成する
date = dt(2016, 1, 1, 16, 0, 0)

# dateの１日後の日時オブジェクトdate_after1daysを作成する
date_after1days = date + timedelta(days=1)

### 1-7.衛星画像を並べて可視化する

**これまでの作業**
```
1.   f文字列の手法による文字列の置換
2.   datetimeモジュールによる日時の操作
```
**複数の画像を並べて表示する**

```
# 1枚目の画像を表示する
plt.subplot(1, 2, 1)
plt.imshow(1枚目の画像)

# 2枚目の画像を表示する
plt.subplot(1, 2, 2)
plt.imshow(2枚めの画像)

# plt.subplot(x, y, z)は、x行y列の図の内、z番目の図に描画することを宣言する
```




In [ ]:
# ライブラリのインポート
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
from datetime import datetime as dt
from datetime import timedelta


# 日時の初期設定として、1つ目の衛星画像に該当する日時を設定する
start_date = dt(2016, 1, 1, 16)

# 5時間分の衛星画像を順番に取得、描画する
for i in range(5):
    
    """
    2. 日時の操作
    """
    # start_dateから、i時間先の日時をdateに代入する
    date = start_date + timedelta(hours=i)
    
    # 年:year、月:month、日:day、時:hourの情報をdateから取得する
    year = date.year
    month = date.month
    day = date.day
    hour = date.hour
    
    # 2018年ならばテストデータ(test)
    # 2016年、2017年ならば学習データ(train)をphaseに代入する
    if year == 2018:
        phase = 'test'
    else:
        phase = 'train'
        
    """
    1. 文字列の置換
    """
    # ファイル名を指定する
    file_path = f"{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png"
    
    # 衛星画像を読み込む
    image = cv2.imread(file_path)
        
    # 1行5列並んだ図の内、i+1番目の図に描画する設定を行う 
    plt.subplot(1, 5, i+1)
    
    # 画像を描画する
    plt.imshow(image)

plt.show()

## 2.気象データの確認・把握

### 2-1.気象データのファイルパス

**各ディレクトリ名についての説明**
```
1.   train と test
学習用(train)として2016年&2017年、評価用(test)として2018年のデータを使用するため、2016年、2017年のデータはtrainディレクトリ内に、2018年のデータはtestディレクトリ内に含まれている。

2.   sat と met
衛星(Satellite)画像データを「sat」気象(Meteorological)データを「met」と、それぞれ表現している。
```

### 2-2.気象データを読み込む

**gzipファイルを扱う方針**
```
1.   ストレージ内に全てのgzipファイルを展開してしまう
# 読み込み処理にかかる時間が削減される一方、ストレージの容量が圧迫される

2.   気象データを読み込むタイミングで毎回gzipファイルを開く
# 読み込み処理にかかる時間がかかる一方、ストレージの容量が圧迫されない
```
**gzipファイルを引数に受け取り、NumPyファイルを返り値とする関数**
```
def Read_gz_Binary(file_path):
    file_tmp = file + "_tmp"
    with gzip.open(file, 'rb') as f_in:
        with open(file_tmp, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    bin_data = np.fromfile(file_tmp, np.float32)
    os.remove(file_tmp)
    return bin_data.reshape( [168,128] )
```

**Read_gz_Binary関数**

```
*   ファイルパスを引数に受け取り
*   形状が(168, 128)のNumPy配列を返り値とする
```
**Read_gz_Binary関数内部の処理**

**1.「gzip.open()」関数によるgzファイルの読み込み**
```
# gzipモジュールのインポート
import gzip

# gzipファイルをバイナリデータで開いたファイルオブジェクトとして返す
with gzip.open(ファイルパス, 'rb') as opened_file:

# 第1引数に、展開したいファイルのファイルパスを指定
# 第2引数は、'r'が読み込み、'b'がバイナリモードを意味する
```
**2.「shutil.copyfileobj()」関数によるファイルオブジェクトのコピー**

```
# shutilのインポート
import shutil

# ファイルオブジェクト1の中身を、ファイルオブジェクト2にコピーする
shutil.copyfileobj(ファイルオブジェクト1, ファイルオブジェクト2)
```
**「np.fromfile()」関数によるバイナリデータの読み込み**
```
# NumPyライブラリのインポート
import numpy as np

# バイナリファイルをNumPy配列として読み込む
bin_data = np.fromfile(ファイルパス, dtypeの指定)

# 第1引数にバイナリファイルのパス
# 第2引数に返り値となるNumPy配列のdtypeを指定する
```
**4. 「np.reshape()」関数による配列の2次元化**
```
# バイナリデータ(1次元の配列)を
# 2次元の配列に変換する
array2d = bin_data.reshape( [168,128] )
```
**Read_gz_Binary()関数の処理の流れ**
```
1.   「gzip.open()」関数によるgzファイルの読み込み
1.   「shutil.copyfileobj()」関数によるファイルオブジェクトのコピー
3.   「np.fromfile()」関数によるバイナリデータの読み込み
```
```
def Read_gz_Binary(file_path):

    # 1. 変数file_tmpに「元のファイル名_tmp」となる文字列を代入
    file_tmp = file_path + "_tmp"

    # 2. gzipファイルを、「読み込み(r)モード & バイナリ(b)モード」 で開き、
    #     開いたファイルオブジェクトをf_inに代入する
    with gzip.open(file_path, 'rb') as f_in:

        # 3. ファイル名を「元のファイル名_tmp」としたファイルを
        #     open()関数の「書き込み(w)モード & バイナリ(b)モード」で新規作成し、
        #     ファイルオブジェクトをf_outとする
        with open(file_tmp, 'wb') as f_out:

            # 4. f_inのファイルオブジェクトをf_outにコピーする
            shutil.copyfileobj(f_in, f_out)

    # 5. バイナリファイルをNumPy配列として読み込む
    met_data = np.fromfile(file_tmp, np.float32)

    # 6. 一時的に作成した展開済みのバイナリファイル「元のファイル名_tmp」を削除する
    os.remove(file_tmp)

    # 7. 読み込んだ配列の形状を(168, 128)に整形する
    met_data = met_data.reshape( [168,128] )

    return met_data
```



In [ ]:
# ライブラリのインポート
import numpy as np
import gzip
import shutil

# 気象データを読み込む関数; Read_gz_Binary を実装する
def Read_gz_Binary(file_path):
    file_tmp = file_path + "_tmp"
    with gzip.open(file_path, 'rb') as f_in:
        with open(file_tmp, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    # バイナリファイルをNumPy配列として読み込む
    met_data = np.fromfile(file_tmp, np.float32)
    os.remove(file_tmp)
    
    # 配列の次元に変更を加える
    met_data = met_data.reshape( [168,128] )
    
    return met_data

# 気象データのファイルパスを指定する
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

# 気象データを読み込む
met_data = Read_gz_Binary(file_path)

# 読み込んだ気象データに関する基本情報を出力する
print(type(met_data))
print(met_data.shape)

### 2-3.気象データを構成する値の統計量を確認する

**算出する統計量**

```
*   最大値
*   最小値
*   平均値
```

In [ ]:
# ライブラリのインポート
import numpy as np

# 気象データのファイルパスを指定する
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

# 気象データを読み込む
met_data = Read_gz_Binary(file_path)

# 最大値の出力
print('最大値: ', np.max(met_data))

# 最小値の出力
print('最小値: ', np.min(met_data))

# 平均値の出力
print('平均値: ', np.mean(met_data))

### 2-4.データの未計測部分を補間する

**1. 北側、南側の未計測部分を補間する(上下方向の補間)**
```
未計測部分
*   北側: 1行目と2行目
*   南側: 155行目〜168行目

有効な計測値
*   北側: 3行目
*   南側: 154行目


# 北側の未計測部分を補間する
data[0:2] = data[2]

# 南側の未計測部分を補間する
data[154:] = data[153]
```

**2. 西側の未計測部分を補間する(左右方向の補間)**
```
未計測部分
*   西側: 1列目〜8列目

有効な計測値
*   西側: 9列目


# 西側の未計測部分を補間する
data[:, :8] = data[:, 8].reshape(-1, 1)
```

In [ ]:
# ライブラリのインポート
import numpy as np

# 気象データのファイルパスを指定する
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

# 気象データを読み込む
met_data = Read_gz_Binary(file_path)

plt.subplot(1, 2, 1)
plt.imshow(met_data, 'gray')

# fill_lack_data()関数を実装する
def fill_lack_data(data):
    
    ## 1. 北側、南側の未計測部分を補間する(上下)
    
    # 北側の未計測部分を補間する
    data[0:2] = data[2]
    # 南側の未計測部分を補間する
    data[154:] = data[153]
    
    
    ## 2. 西側の未計測部分を補間する(左右)
    
    # 西側の未計測部分を補間する
    data[:, :8] = data[:, 8].reshape(-1, 1)
    
    return data

# fill_lack_data()関数をmet_dataに対して実行する
filled_data = fill_lack_data(met_data)



plt.subplot(1, 2, 2)
plt.imshow(filled_data, 'gray')

plt.show()

### 2-5.ファイルパスを一般化する

**構成成分の確認**
```
{train もしくは test}/met/{年}/{月}/{日}/{気象データの名称}.3.{年}{月}{日}{時}.gz

衛星画像データと同じく
*   train もしくは test
*   年
*   月
*   日
*   時
の5つに加えて、
*   気象データの名称
```
**気象データの種類名**
```
*   アルファベット名: 気象データの種類
*   数字: 等圧面における気圧


*   各アルファベット名と各種気象データの対応
アルファベット名	気象データの種類
HGT	高度
PRMSL	海面気圧
RH	湿度
TMP	気温
UGRD	東西風
VGRD	南北風
VVEL	鉛直流
```
**それぞれの変数の対応**
```
*   phase: train もしくは test
*   year: 年
*   month: 月
*   day: 日
*   hour: 時
*   data_type: 気象データの名称
```

In [ ]:
# ライブラリのインポート
from datetime import datetime as dt

# 日時オブジェクトの作成
date = dt(2016, 1, 1, 18)

"""
file_path = 'train/met/2016/01/01/HGT.200.3.2016010118.gz'

となるように、以下の変数phase, year, month, day, hour, data_typeに値を代入しましょう。
"""

phase = 'train'
year = date.year
month = date.month
day = date.day
hour = date.hour
data_type = 'HGT.200'



# f文字列を利用して、ファイルパスを表す文字列を作成する
file_path = f'{phase}/met/{year}/{month:02}/{day:02}/{data_type}.3.{year}{month:02}{day:02}{hour:02}.gz'


print(file_path)

# [2] データセットの作成

## 1.データが与えられている日付の取得

### 1-1.学習用データが与えられている日付を取得する

**日時の操作**
```
# datetime timedeltaのインポート
from datetime import datetime as dt
from datetime import timedelta

# 日時データの作成 「2016年1月1日16時」
date = dt(2016, 1, 1, 16)

# 日時データの演算 (1日後)
date_after1day = date + timedelta(days=1)
```



In [ ]:
# ライブラリのインポート
from datetime import datetime as dt
from datetime import timedelta

# 学習データが提供されている日の日付を取得する
def get_train_days_list():
    
    # 2016年１月１日１６時からスタート
    start_date = dt(2016, 1, 1, 16)
    
    # 日付を格納するリストを初期化する
    days_list = []
    
    # 学習データは365日×２年分与えられるため、合計730日分の日時を取得する
    for i in range(2 * 365):
        
        # start_dateから、i日後の日時を取得し、dateに代入する
        date = start_date + timedelta(days=i)
        
        # days_listにdateを追加する
        days_list.append(date)
        
    return days_list

# 関数を実行して、学習データが提供されている日の日付を取得する
train_days_list = get_train_days_list()



print(train_days_list[:5])

### 1-2.テスト用データが与えられている日付を取得する

**文字列を日付データに変換する**
```
import pandas as pd

# OpenData_startカラム内の全レコードを日付型のデータに変換
test_terms['OpenData_start'] = pd.to_datetime(test_terms['OpenData_start'])
```



In [ ]:
# ライブラリのインポート
import pandas as pd

# テスト対象期間の日付を取得する関数
def get_test_days_list():
    # test_terms.csvからtest対象期間の日付を取得
    test_terms = pd.read_csv('test_terms.csv')
    
    # OpenData_startカラムのデータを日付型のデータとして取得する
    days_list = pd.to_datetime(test_terms['OpenData_start'])
            
    return days_list

test_days_list = get_test_days_list()
print(test_days_list[:5])

## 2.衛星画像データをまとめたnpyファイルの作成

### 2-1.衛星画像データの読み込みを行う関数を実装する

**衛星画像データを読み込む関数get_sat_data()**
```
引数: 日時
返り値: 衛星画像データを表すNumPy配列
```
**ファイルパスの特定**
```
# 衛星画像データのファイルパス
file_name = f"{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png"
```
**一般化されたファイルパスに含まれる変数**
```
*   phase
*   year
*   month
*   day
*   hour
```

In [ ]:
# 衛星画像データを読み込む関数get_sat_dataの作成
def get_sat_data(date):
    
    # 日付(date)から、年(year)、月(month)、日(day)、時(hour)を数値データとして取得する
    year = date.year
    month = date.month
    day = date.day
    hour = date.hour
    
    # 2018年ならばテストデータ(test)
    if year == 2018:
        phase = 'test'
    # 2016年、2017年ならば学習データ(train)
    else:
        phase = 'train'
        
    # ファイル名を指定する
    file_name = f"{phase}/sat/{year}-{month:02}-{day:02}/{year}-{month:02}-{day:02}-{hour:02}-00.fv.png"
    
    # 画像の読み込み
    sat_data = cv2.imread(file_name, 0)
    
    return sat_data

# 「2016年1月1日16時」時点の衛星画像を読み込む
sat_data = get_sat_data(date=dt(2016, 1, 1 ,16))


print(sat_data)

### 2-2.npyファイル作成の方針(衛星画像データ)

**衛星画像データを構成する5次元配列**
```
(サンプル数(対象日の日数), 時系列の長さ(24時間), 高さ, 幅, チャンネル数)
```
**1. 対象日の日数の違いについて**
```
学習用データ: 2016年、2017年の合計730日分
テストデータ: 2018年内から選ばれた50日分
```
**2. 画像の高さ、幅について**

```
画像分類タスクで使用されることの多いデータセットの画像サイズの例

*   ImageNet: (224, 224)
*   CIFAR-10: (32, 32)
```

### 2-3.衛星画像データをまとめたnpyファイルを作成する関数

**make_sat_dataset()**

```
# 引数resizeとphaseを設定
*   オリジナルの画像を何分の1サイズに縮小するか(resize)
*   学習用データセットを作成するかorテスト用データセットを作成するか(phase)
```
**データセット作成時の詳しい処理**
```
def make_sat_dataset(resize, phase):
    # 引数phaseの値に従って、
    # 学習用、もしくはテスト用のデータが与えられている日付を取得する
    ## [get_train_days_list()、get_test_days_list()はともにタスク1で作成した関数]
    if phase == 'train':
        start_date_list = get_train_days_list()
    elif phase == 'test':
        start_date_list = get_test_days_list()

    # データセットを格納するリストの初期化
    dataset = []

    # start_date_listから、順番にデータが与えられている日付を取得し、
    # 変数start_dateに代入する
    for start_date in start_date_list:

        # 一日分の衛星画像データを格納するリストの初期化
        data_in_1day = []

        # 24時間分の衛星画像を順番に取得する
        for i in range(24):
            # start_dateから、時間を「i時間」分進める
            date = start_date + timedelta(hours=i)

            # 与えられた日時(date)における衛星画像データを読み込む
            ## [get_sat_data()関数は当タスク内で作成した関数]
            sat_data = get_sat_data(date)

            # 引数resizeの値に従って、データを縮小する
            resized_data = cv2.resize(sat_data, ( int(512 / resize), int(672 / resize) ), 
                                                          interpolation=cv2.INTER_AREA)

            # data_in_1dayリストへの値の追加
            data_in_1day.append(resized_data)

        # 1日分(24時間分)のデータを格納したリストをNumPy配列に変換する
        # 形状は(24, 高さ, 幅, 1)
        data_in_1day = np.array(data_in_1day, dtype='uint8').reshape(24, int(672 / resize), int(512 / resize), 1)

        # 取得した1日分のNumPy配列をdatasetリストに追加する
        dataset.append(data_in_1day)

    # 対象期間に含まれる全データを格納したリストをNumPy配列に変換する
    # 形状は(対象期間の日数, 24, 高さ, 幅, 1)
    dataset = np.array(dataset, dtype='uint8').reshape(len(start_date_list), 24, int(672 / resize), int(512 / resize), 1)

    # 保存するnpyファイル名を指定する
    save_name = f'{phase}_sat.npy'

    # NumPy配列をnpyファイルとして保存する
    np.save(save_name, dataset)
```


**cv2.resize()関数: 画像のリサイズ**
```
# 画像のリサイズ
resized_image = cv2.resize(画像を表す配列, (リサイズ後の画像の幅, リサイズ後の画像の高さ), interpolation=cv2.INTER_AREA)
```
**np.save(): NumPy配列をファイルとして保存する**
```
# NumPy配列をファイルとして保存する
np.save(ファイル名, 保存の対象となるNumPy配列)
```


In [ ]:
def make_sat_dataset(resize, phase):
    
    if phase == 'train':
        # 学習用データが与えられている日付のリストを取得する
        start_date_list = get_train_days_list()
    elif phase == 'test':
        # テスト用データが与えられている日付のリストを取得する
        start_date_list = get_test_days_list()

    dataset = []
        
    for start_date in start_date_list:
        
        data_in_1day = []
        
        for i in range(24):
            date = start_date + timedelta(hours=i)
            sat_data = get_sat_data(date)
            
            # データの縮小を実行する
            resized_data = cv2.resize(sat_data, ( int(512 / resize), int(672 / resize) ), 
                                                          interpolation=cv2.INTER_AREA)
            data_in_1day.append(resized_data)
            
        data_in_1day = np.array(data_in_1day, dtype='uint8').reshape(24, int(672 / resize), int(512 / resize), 1)
        dataset.append(data_in_1day)
        
    dataset = np.array(dataset, dtype='uint8').reshape(len(start_date_list), 24, int(672 / resize), int(512 / resize), 1)
    save_name = f'{phase}_sat.npy'
    
    # npyファイルの保存を実行する
    np.save(save_name, dataset)
    
    
# 「縦横20分の1サイズに縮小した」 「学習用衛星画像データ」を
# npyファイルとしてまとめて保存する 
make_sat_dataset(resize=20, phase='train')



sat_dataset = np.load('train_sat.npy')

## 3.気象データをまとめたnpyファイルの作成

### 3-1.気象データを読み込む関数を実装する

**与えられた日時における気象データを読み込む関数**

In [ ]:
# 「日付(date)」と「気象データの種類(data_type)」を指定して、気象データを読み込む関数
def get_met_data(date, data_type):
    
    # 日付(date)から、年(year)、月(month)、日(day)、時(hour)を数値データとして取得する
    year = date.year
    month = date.month
    day = date.day
    hour = date.hour
    
    # 2018年ならばテストデータ(test)
    if year == 2018:
        phase = 'test'
    # 2016年、2017年ならば学習データ(train)
    else:
        phase = 'train'
        
    # ファイル名を指定する
    file_name = f'{phase}/met/{year}/{month:02}/{day:02}/{data_type}.3.{year}{month:02}{day:02}{hour:02}.gz'
    
    # データの読み込み(Read_gz_Binary) -> 空白箇所の穴埋め(fill_lack_data)の順番で処理を行う
    met_data = fill_lack_data(Read_gz_Binary(file_name))
    
    return met_data

# get_met_data()関数を実行し、
# 2016年1月1日18時の「'HGT.200'」に関する気象データを読み込む
met_data = get_met_data(date=dt(2016, 1, 1, 18), data_type='HGT.200')

### 3-2.npyファイル作成の方針(気象データ)

**大まかな処理の流れ**
```
1.   データが3時間ごとに提供されている
2.   データが34種類に分けて提供されている
```
**1. データが3時間ごとに提供されていることへの対応**
```
*   衛星画像データ: 1時間ごと
*   気象データ: 3時間ごと

# 3時間間隔で提供されている気象データを、間に生じた隙間を補間することで1時間間隔のデータに変える
```
**2. データが34種類に分けて提供されていることへの対応**
```
# 気象データの種類を表す文字列を格納したリスト
met_data_type_list = ['HGT.200', 'HGT.300', 'HGT.500', 'HGT.700', 'HGT.850', 
                         'PRMSL.msl', 
                         'RH.1p5m', 'RH.300', 'RH.500', 'RH.700', 'RH.850', 
                         'TMP.1p5m', 'TMP.200', 'TMP.300', 'TMP.500', 'TMP.700', 'TMP.850',
                         'UGRD.10m','UGRD.200', 'UGRD.300', 'UGRD.500', 'UGRD.700', 'UGRD.850', 
                         'VGRD.10m', 'VGRD.200', 'VGRD.300', 'VGRD.500', 'VGRD.700', 'VGRD.850', 
                         'VVEL.200', 'VVEL.300', 'VVEL.500', 'VVEL.700', 'VVEL.850']
```
```
# forループでリストに含まれる文字列を順に読み込む
for data_type in met_data_type_list:

    # 読み込んだ引数をget_met_data関数の引数として受け渡し、
    # 該当する気象データを読み込む
    met_data = get_met_data(date=date, data_type=data_type)

    ### 以下にひとつひとつの気象データに対して実行する処理を記述する
```

### 3-3.気象データをまとめたnpyファイルを作成する

**関数内の大まかな処理の流れ**

```
resize: オリジナルの画像を何分の1サイズに縮小するか
phase: 学習用データを作成するかorテスト用データを作成するか
```
```
# 気象データの種類を表す文字列を格納したリスト
met_data_type_list = ['HGT.200', 'HGT.300', 'HGT.500', 'HGT.700', 'HGT.850', 
                     'PRMSL.msl', 
                     'RH.1p5m', 'RH.300', 'RH.500', 'RH.700', 'RH.850', 
                     'TMP.1p5m', 'TMP.200', 'TMP.300', 'TMP.500', 'TMP.700', 'TMP.850',
                     'UGRD.10m','UGRD.200', 'UGRD.300', 'UGRD.500', 'UGRD.700', 'UGRD.850', 
                     'VGRD.10m', 'VGRD.200', 'VGRD.300', 'VGRD.500', 'VGRD.700', 'VGRD.850', 
                     'VVEL.200', 'VVEL.300', 'VVEL.500', 'VVEL.700', 'VVEL.850']
```
**HGT(高度)に関する特徴量のみを採用**
```
# 高度に関する気象データのみを抽出したリスト
HGT_list = ['HGT.200', 'HGT.300', 'HGT.500', 'HGT.700', 'HGT.850']
```
**関数make_met_dataset()**
```
def make_met_dataset(resize, phase, data_type_list):

    if phase == 'train':
        start_date_list = train_days_list
    elif phase == 'test':
        start_date_list = get_test_days_list()

    # 全種類、全日時分のデータを格納するリストの初期化
    dataset = []

    for start_date in start_date_list:

        # 引数data_type_listから、
        # 気象データの種類を示す文字列を1つずつ取り出す。
        for count, data_type in enumerate(data_type_list):

            # 1種類、1日分のデータを格納するリストの初期化
            one_type_in_1day = []

            for i in range(24):

                date = start_date + timedelta(hours=i)
                hour = date.hour

                """データ補間処理の方針
                A: 
                3で割り切ることのできる時刻T(0時, 3時, 6時, 9時, 12時, 15時, 18時, 21時)は、
                気象データが提供されているため、単純にデータを読み込む。

                B: 
                それ以外の時刻T+1とT+2については、
                時刻Tと時刻T+3時点の気象データを利用して、値を補間する。

                """

                # 時刻がT[0, 3, 6, 9, 12, 15, 18, 21]時の場合
                if hour % 3 == 0:

                    met_data = get_met_data(date, data_type)

                # 時刻がT+1[1, 4, 7, 10, 13, 16, 19, 22]時の場合
                elif hour % 3 == 1:

                    # １時間前(時刻T)の気象データを読み込む
                    before_1h = date - timedelta(hours=1)
                    data_before_1h = get_met_data(before_1h, data_type)

                    # ２時間後(時刻T+3)の気象データを読み込む
                    after_2h = date + timedelta(hours=2)
                    data_after_2h = get_met_data(after_2h, data_type)

                    # 前後の値を利用して、補間する
                    met_data = (2/3) * data_before_1h + (1/3) * data_after_2h

                # 時刻がT+2[2, 5, 8, 11, 14, 17, 20, 23]時の場合
                else:

                    # ２時間前(時刻T)の気象データを読み込む
                    before_2h = date - timedelta(hours=2)
                    data_before_2h = get_met_data(before_2h, data_type)

                    # １時間後(時刻T+3)の気象データを読み込む
                    after_1h = date + timedelta(hours=1)
                    data_after_1h = get_met_data(after_1h, data_type)

                    # 前後の値を利用して、補間する
                    met_data = (1/3) * data_before_2h + (2/3) * data_after_1h

                # 引数resizeの値に従って、データを縮小する
                resized_data = cv2.resize(met_data, (int(512 / resize), int(672 / resize)),
                                                              interpolation=cv2.INTER_AREA)

                # 1種類、1時刻分のデータをone_type_in_1dayリストへ追加する
                one_type_in_1day.append(resized_data)

            one_type_in_1day = np.array(one_type_in_1day).reshape(24, int(672 / resize), int(512 / resize), 1)

            # 1種類目はall_type_in_1dayに代入する
            if count == 0:
                all_type_in_1day = one_type_in_1day
            # 2種類目以降はall_type_in_1dayにチャンネルの次元で結合する
            else:
                all_type_in_1day = np.concatenate([all_type_in_1day, one_type_in_1day], axis=3)

        # 全種類、1日分のデータをdatasetリストへ追加する
        dataset.append(all_type_in_1day)

    dataset = np.array(dataset, dtype='float32').reshape(len(start_date_list), 24, int(672 / resize), int(512 / resize), len(data_type_list))
    save_name = f'{phase}_met.npy'
    np.save(save_name, dataset)
```





In [ ]:
def make_met_dataset(resize, phase, data_type_list):

    if phase == 'train':
        start_date_list = get_train_days_list()
    elif phase == 'test':
        start_date_list = get_test_days_list()        
        
    dataset = []    
    
    for start_date in start_date_list:
        
        for count, data_type in enumerate(data_type_list):
        
            one_type_in_1day = []
            
            for i in range(24):
                
                date = start_date + timedelta(hours=i)
                hour = date.hour
                
                # セクションA #
                if hour % 3 == 0:
                    met_data = get_met_data(date, data_type)
                elif hour % 3 == 1:
                    before_1h = date - timedelta(hours=1)
                    data_before_1h = get_met_data(before_1h, data_type)
                    after_2h = date + timedelta(hours=2)
                    data_after_2h = get_met_data(after_2h, data_type)
                    met_data = (2/3) * data_before_1h + (1/3) * data_after_2h
                else:
                    before_2h = date - timedelta(hours=2)
                    data_before_2h = get_met_data(before_2h, data_type)
                    after_1h = date + timedelta(hours=1)
                    data_after_1h = get_met_data(after_1h, data_type)
                    met_data = (1/3) * data_before_2h + (2/3) * data_after_1h
                # セクションA #
               
                resized_data = cv2.resize(met_data, (int(512 / resize), int(672 / resize)),
                                                              interpolation=cv2.INTER_AREA)
                one_type_in_1day.append(resized_data)
                     
            one_type_in_1day = np.array(one_type_in_1day).reshape(24, int(672 / resize), int(512 / resize), 1)
            
            # セクションB #
            if count == 0:
                all_type_in_1day = one_type_in_1day
            else:
                all_type_in_1day = np.concatenate([all_type_in_1day, one_type_in_1day], axis=3)
            # セクションB #
                
        dataset.append(all_type_in_1day)
    
    # セクションC #
    dataset = np.array(dataset, dtype='float32').reshape(len(start_date_list), 24, int(672 / resize), int(512 / resize), len(data_type_list))
    save_name = f'{phase}_met.npy'
    np.save(save_name, dataset)
    # セクションC #

# [3] 前処理

## 1.学習データと検証データの分割

### 1-1.学習データと検証データの分割

**[2]で作成した4つのnpyファイル**
```
説明	ファイル名	形状
学習用 衛星画像データ	train_sat.npy	(730, 24, 33, 25, 1)
テスト用 衛星画像データ	test_sat.npy	(50, 24, 33, 25, 1)
学習用 気象データ	train_met.npy	(730, 24, 33, 25, 34)
テスト用 気象データ	test_met.npy	(50, 24, 33, 25, 34)
```
**npyファイルの読み込み方**
```
# npyファイルの読み込み
train_sat_dataset = np.load('train_sat.npy')
```

In [ ]:
# ライブラリのインポート
import numpy as np

# train_sat.npyを読み込む
train_sat_dataset = np.load('train_sat.npy')

# 学習用データを抽出する
train_sat = train_sat_dataset[:365]

# 検証用データを抽出する
val_sat = train_sat_dataset[365:]


print(train_sat.shape)
print(val_sat.shape)

## 2.データの正規化

### 2-1.衛星画像データの正規化を行う

**正規化の方針**
```
# 全ての値を0以上1以下の間に収める手法

# 衛星画像データを255で割る
train_sat = train_sat / 255
```
**データ型を変更**
```
# uint8型のデータをfloat32型のデータに変換
train_sat = train_sat.astype(np.float32)
```
**補足) float32型を採用する理由**
```
float32型とfloat64型は、
1.   数値表現の精密さ
2.   メモリサイズ
のトレードオフ
```

In [ ]:
# 衛星画像データを正規化する
train_sat = train_sat.astype(np.float32) / 255



print(train_sat.dtype)
print(train_sat.max())
print(train_sat.min())

### 2-2.気象データの正規化を行う① 最大値、最小値の取得

**気象データを種類別に抽出する**
```
# 1種類目の気象データのみを抽出する
train_met_0 = train_met[:, :, :, :, 0]

"""
形状が(730, 24, 33, 25, 34)であるtrain_metに対して、
5つ目の次元でインデックス0を指定することで、1種類目の気象データのみが抽出され、
train_met_0は、形状(730, 24, 33, 25)となる
"""
```

In [ ]:
# 気象データの最大値、最小値を取得する関数get_met_max_min()
def get_met_max_min(train_met):
    # 気象データのチャンネル数34を取得する
    n_data_type = train_met.shape[4]
    
    max_array = []
    min_array = []
    
    # 0から33までのインデックスを順番に取得する
    for i in range(n_data_type):
    
        # チャンネルの次元をインデックスiで指定し、
        # チャンネルごとの最大値、最小値を取得する
        max_value = np.max(train_met[:, :, :, :, i])
        min_value = np.min(train_met[:, :, :, :, i])
        
        max_array.append(max_value)
        min_array.append(min_value)
        
    max_array = np.array(max_array)
    min_array = np.array(min_array)
    
    return max_array, min_array


# get_met_max_min()関数を実行する
max_array, min_array = get_met_max_min(train_met=train_met)

print('各種気象データの最大値を格納した配列', max_array)
print('--------------------------------------------------------------')
print('各種気象データの最小値を格納した配列', min_array)

### 2-3.気象データの正規化を行う② 正規化の実行

**最大値、最小値を利用して、実際に各種の気象データの正規化**
```
normalize_met_data()関数

引数として、
met_data: 正規化前の気象データ
max_array: 最大値を格納した配列
min_array: 最小値を格納した配列
を受け取り、返り値が正規化済みのNumPy配列となるように設計
```
**正規化の公式**
```
# 正規化の公式
正規化後のデータ = (元のデータ - 最小値) / (最大値 - 最小値)
```

In [ ]:
# 気象データを[0, 1]の開区間に正規化する関数normalize_met_data
def normalize_met_data(met_data, max_array, min_array):

    n_data_type = met_data.shape[4]
    
    for i in range(n_data_type):
        # max_array, min_arrayを利用して、正規化を実行する
        met_data[:, :, :, :, i] = (met_data[:, :, :, :, i] - min_array[i]) / (max_array[i] - min_array[i])
        
    return met_data
    

train_met = normalize_met_data(met_data=train_met, 
                               max_array=max_array, min_array=min_array)       

## 3.入力データと正解データの分割

### 3-1.入力データと正解データの分割

**rain_sat val_satの分割**
```
学習データと検証データの現在の形状
*   train_sat: (365, 24, 33, 25, 1)
*   val_sat: (365, 24, 33, 25, 1)

インデックス指定でデータを抽出することで、同じインデックスが示すデータが、『入力』&『正解』の関係として成り立つように分割
*   入力データのインデックス:0〜363
*   正解データのインデックス:1〜364
```
**インデックス配列の作成**

```
*   入力:0〜363
*   正解:1〜364
```
**np.arange()関数の引数**

```
*   第1引数: start
*   第2引数: stop
*   第3引数: step

*   区間が[start, stop)
*   公差(等差数列における隣の項との差)がstep

# 0から10までの整数を並べる
array = np.arange(0, 11, 1)

np.arange()関数が返り値とする等差数列は、
*   start以上(start <= x)
*   stop未満(x < stop)
```

**補足) np.arange()関数の省略**
```
1.   位置引数を2つのみ指定した場合
# 0から10までの整数を並べる
array = np.arange(0, 11)

"""
位置引数は
start = 0
stop = 11
を指定したと判断される

stepはデフォルトの1

"""


2.   位置引数を1つのみ指定した場合
# 0から10までの整数を並べる
array = np.arange(11)

"""
位置引数は
stop = 11
を指定したと判断される

startはデフォルトの0
stepはデフォルトの1

"""

```

In [ ]:
# 入力データのインデックス配列をX_indexに代入する
X_index = np.arange(364)

# 正解データのインデックス配列をY_indexに代入する
Y_index = np.arange(1, 365)


# インデックスに従って、入力データ、正解データを抽出する
X_train = train_sat[X_index]
Y_train = train_sat[Y_index]

# [4] モデリング

## 1.Convolutional LSTMモデルの構築

### 1-1.Convolutional LSTM層を実装する

**Convolutional LSTMとは**
```
*   CNN(Convolutional Neural Network) 画像データの処理
*   LSTM(Long short-term memory) 時系列データの処理
という2つのネットワークを組み合わせることによって生まれた応用的なネットワーク
```
**KerasにおけるConvolutional LSTMの実装方法**
```
Tensorflow.keras
# 一般的なニューラルネットワーク層
*   Conv2D
*   Dense
*   LSTM

# Convolutional LSTM層

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D

# モデルを初期化する
model = Sequential()

# ConvLSTM層の追加
model.add(ConvLSTM2D())
```
**ConvLSTM2D層に指定できる引数**
```
引数名	引数に代入する値	引数の説明
filters	20	ConvLSTM2Dが出力するフィルタ(チャンネル)の数
kernel_size	(3, 3)	畳み込みに使用するウィンドウのサイズ
padding	'same'	畳み込み後、画像サイズを入力サイズに合わせるための補間を実行するか否かの指定
activation	'tanh'	ConvLSTM層の出力時に使用する活性化関数
recurrent_activation	'sigmoid'	各再帰(LSTM)セルの出力時に使用する活性化関数
input_shape	(24, 33, 25, 1)	バッチサイズを除いた入力の形状。入力層のみ指定することが必須となっている
return_sequences	True	Trueならば、出力された配列をそのまま返す。Falseならば、時系列方向で最後に出力された配列のみ返す。
```

In [ ]:
# ライブラリのインポート
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D

# Sequentialのインスタンスを生成することで、モデルの雛形とする
model = Sequential()

# ConvLSTM2D層を実装する
model.add(ConvLSTM2D(filters=20, kernel_size=(3, 3), padding='same', return_sequences=True,
                                            activation='tanh', recurrent_activation='sigmoid',
                                            input_shape=(24, 33, 25, 1)))

print(model.summary())

### 1-2.Encoder、Decoderを実装する

**seq2seqを導入する目的**
```
時系列データの生成を可能にするため

例えば、本来1時間後までしか予測することのできないモデルを、一度に24時間後まで予測できるようにする
```
**seq2seqの構成要素: EncoderとDecoder**
```
*   Encoder: 入力データの情報を圧縮する
*   Decoder:Encoderによって圧縮されたデータを展開し、出力とする
```
**Encoder、Decoderの実装上の違い: 「return_sequences」**
```
*   Trueを指定すると、時系列方向について、全ての配列が出力となる
*   Falseを指定すると、時系列方向について、最後の時点の配列のみが出力となる
```
**Kerasにおいて実装する場合**
```
*   Encoder層はreturn_sequences=False
*   Decoder層はreturn_sequences=True
```
**Encoderにおいて、return_sequencesをFalseとする理由**
```
RNN系(再帰型)のネットワークは、過去(t-1)の出力データを次の時点(t)の入力データになんらかの方法で受け渡すという工程を繰り返すことで、過去の状態を考慮した予測が可能になるというアルゴリズム。

つまり、入力データの時系列方向のサイズをTとすると、T番目の出力(時系列方向で最後となる出力)は、1, 2, 3, .... , T- 2, T-1までの過去の情報を全て保持していると考えることができる。
よって、入力データの圧縮が目的であるEncoder層は、過去の状態を全て保持したT番目の出力(時系列方向で最後の出力)のみを必要とし、1, 2, 3, ...., T-2, T-1の情報は出力から除外する。
```







In [ ]:
# ライブラリのインポート
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D

# Encoder層を実装する
Encoder = Sequential()
Encoder.add(ConvLSTM2D(
                    filters=20, kernel_size=(3, 3), padding='same', return_sequences=False,
                    activation='tanh', recurrent_activation='sigmoid',
                    input_shape=(24, 33, 25, 1)))

# Decoder層を実装する
Decoder = Sequential()
Decoder.add(ConvLSTM2D(
                    filters=20, kernel_size=(3, 3), padding='same', return_sequences=True,
                    activation='tanh', recurrent_activation='sigmoid',
                    input_shape=(24, 33, 25, 20)))


print(Encoder.summary())
print(Decoder.summary())

### 1-3.EncoderとDecoderをつなぐ

**現状**

```
Encoder入力前: (バッチサイズ, 24, 高さ, 幅, チャンネル数)
Encoder出力後: (バッチサイズ, 高さ, 幅, チャンネル数)

# 入力データが5次元の配列であることを必要とするDecoderのConvLSTM2Dに入力しようとするとエラーが発生してしまう


1.   時系列方向の次元を復活させる
(バッチサイズ, 高さ, 幅, チャンネル数) -> (バッチサイズ, 1, 高さ, 幅, チャンネル数)
2.   配列をコピーし、時系列方向に連結することで、予測したい時系列の長さ(当クエストの場合、24)に揃える
(バッチサイズ, 1, 高さ, 幅, チャンネル数) -> (バッチサイズ, 24, 高さ, 幅, チャンネル数)
```
**モデル内で任意の処理を実行する: Lambda層**

```
# Lambda層のインポート
from tensorflow.keras.layers import Lambda

# Lambda層の追加
model.add(Lambda(repeat_last_status))
```
**Lambdaの引数となる関数repeat_last_status**
```
*   引数として、前の層から出力されたTensorを受け取り
*   返り値として、次の層に入力するTensorを設定する
```


**tf.Tensor型**

```
# Tensorflowライブラリのインポート
import tensorflow as tf
```
**1. 時系列方向の次元を復活させる(tf.reshape)**

```
# tf.reshape()関数による形状の変更
x = tf.reshape(x, (-1, 1, 33, 25, 20))
# -1と指定した次元の成分数は、他の次元に指定した値から推測され、自動的に決まる
```
**2. 配列をコピーし(tf.identity)、時系列方向に連結することで(tf.concat)、予測したい時系列の長さに揃える**

```
# tf.Tensor型のデータをコピーし、新しいオブジェクトとして生成する
copied_x = tf.identity(x)


# 時系列方向の次元(2つ目の次元)で配列を連結する
x = tf.concat([x, copied_x], axis=1)
```

In [ ]:
# ライブラリのインポート
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda

def repeat_last_status(x):
    
    # 1. 時系列方向の次元を復活させる
    x = tf.reshape(x, (-1, 1, 33, 25, 20))
    
    # tf.Tensor型のデータをコピーし、新しいオブジェクトとして生成する
    copied_x = tf.identity(x)
    
    # 時系列方向を軸とした連結を23回繰り返す
    for _ in range(23):
        
        x = tf.concat([x, copied_x], axis=1)
        
    return x

# Sequentialのインスタンスを生成することで、モデルの雛形とする
model = Sequential()
# Lambda層を実装する
model.add(Lambda(repeat_last_status))


# 次元数が4の配列xを生成
x = tf.ones((10, 33, 25, 20))

# repeat_last_statusを実行する
repeated_x = repeat_last_status(x)

print('次元数が4の配列x:')
print(x.shape)
print('-----------------------------------------------')
print('repeat_last_statusによって変換後の配列x:')
print(repeated_x.shape)

### 1-4.出力層を実装する

**Decoderによって出力された配列に対して、「1x1サイズのカーネル」によって、畳み込みを行う層を追加**

**TimeDistributed**

```
配列を時系列の軸で分割し、分割後の配列それぞれに対して、演算を行う

# TimeDistributedのインポート
from tensorflow.keras.layers import TimeDistributed, Conv2D

# TimeDistributedを用いた層の追加
model.add(TimeDistributed(Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')))
```
**出力層の演算: Conv2Dについて**

```
1. filters(フィルター数)
Conv2Dのフィルター数(filtes)は1設定することで、出力される配列の形状がY(正解データ)と同じく、(バッチサイズ, 高さ, 幅, 1)となるようにする

2. activation(活性化関数)
Conv2Dのactivation(活性化関数)は、シグモイド関数(sigmoid)を選択する
```


```
Conv2Dの引数名	引数に代入する値
filters	1
kernel_size	(1, 1)
activation	'sigmoid'
```





In [ ]:
# ライブラリのインポート
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D

# Sequentialのインスタンスを生成することで、モデルの雛形とする
model = Sequential()

# TimeDistributedでラップされたConv2Dを実装する
model.add(TimeDistributed(Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid'),
                         input_shape=(24, 33, 25, 20)))

### 1-5.損失関数と最適化手法を設定する

**compileメソッド**

```
"""例)
損失関数に交差エントロピー
最適化手法に確率的勾配降下法を指定する場合
"""

# 損失関数、最適化手法の指定
model.compile(loss='categorical_crossentropy', optimizer='sgd')
```
**1. 損失関数の選択**
```
MAE(平均絶対誤差: Mean Absolute Error)を採用する
```
**2. 最適化手法の選択**

```
Adamを選択する

Adamは、
*   学習率: learning_rate
*   モーメンタム: momentum
といった最適化手法に関するパラメータを特にいじることなく、デフォルトの設定のままで学習を実行しても一定の成功が保証されている使い勝手の良い最適化手法
```

In [ ]:
model = Sequential()

# Encoder
model.add(ConvLSTM2D(
                    filters=20, kernel_size=(3, 3), 
                    padding='same', return_sequences=False,
                    activation='tanh', recurrent_activation='sigmoid',
                    input_shape=(24, 33, 25, 1)))

# EncoderとDecoderをつなぐ中間部分
def repeat_last_status(x):
    x = tf.reshape(x, (-1, 1, 33, 25, 20))
    copied_x = tf.identity(x)
    for _ in range(23):
        x = tf.concat([x, copied_x], axis=1)        
    return x
model.add(Lambda(repeat_last_status))

# Decoder
model.add(ConvLSTM2D(
                    filters=20, kernel_size=(3, 3),
                    padding='same', return_sequences=True,
                    activation='tanh', recurrent_activation='sigmoid'))          

# 出力層
model.add(TimeDistributed(Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')))
          

# モデルをコンパイルする
model.compile(loss='mae', optimizer='adam')

## 2.コールバックの設定

### 2-1.コールバックとは

```
コールバックは、主にモデルの学習の進行状況に応じて、特定の処理を行うことのできる機能の集まり

使用頻度の高いものとして、
*   EarlyStopping: 評価スコアを監視し、モデルの改善が止まったタイミングで学習の早期終了を行う
*   ModelCheckpoint: 各エポック終了時のモデルの状態を保存する
*   Tensorboard: 可視化ツール「Tensorboard」の使用に必要なログを保存する
などがある。
```


### 2-1.EarlyStoppingを実装する

**1. EarlyStoppingのインポート**

```
# EarlyStoppingのインポート
from tensorflow.keras.callbacks import EarlyStopping
```
**2. EarlyStoppingのインスタンス生成**

```
EarlyStopping(早期終了)を実行するには、どういう基準で学習を早期に終わらせるのかという点を明らかにする必要

・ EarlyStoppingのインスタンスを生成する際の引数
引数名	引数の説明
monitor	モデルの学習が上手くいっているのか否かの判定に使用する評価基準の指定
patience	monitorに設定した評価スコアの改善が止まって以降何エポック後に学習を終了させるかの指定

# monitorに'val_loss', patienceに30を指定してインスタンス生成
e_stopping = EarlyStopping(monitor='val_loss', patience=30)
```




In [ ]:
# EarlyStoppingのインポート
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStoppingのインスタンスを生成する
e_stopping = EarlyStopping(monitor='val_loss', patience=10)

### 2-3.ModelCheckpointを実装する


```
ModelCheckpointは、各エポック終了時のモデルの状態をファイルとして保存する役割
```
**1. ModelCheckpointのインポート**
```
# ModelCheckpointのインポート
from tensorflow.keras.callbacks import ModelCheckpoint
```
**2. ModelCheckpointのインスタンス生成**
```
ModelCheckpointのインスタンス生成の際の引数
引数名	引数の説明
filepath	モデルの状態を保存するファイル名の指定
monitor	モデルの学習が上手くいっているのか否かの判定に使用する評価基準
save_best_only	Trueとした場合、モデルのmonitorの値がベストスコアを更新した場合のみモデルの保存を実行する
save_weights_only	Trueとした場合、パラメータのみを保存の対象とし、モデルの構造などその他の状態の保存は行わない

# ModelCheckpointクラスのインスタンス生成
checkpoint = ModelCheckpoint(filepath='my_params.h5',
                             monitor='val_loss', save_best_only=True, save_weights_only=True)
```
**filepathの柔軟な変更**
```
置換フィールド{}に指定することのできる主な変数名
変数名	変数の説明
epoch	第何エポック終了時点であるのかを示す整数
loss	エポック終了時点における学習データに対する損失値
val_loss	エポック終了時点における検証データに対する損失値

# ModelCheckpointクラスのインスタンス生成
checkpoint = ModelCheckpoint(filepath='my_params_Epoch{epoch}_loss{loss:.4f}_valloss{val_loss:.4f}.h5',
                             monitor='val_loss', save_best_only=True, save_weights_only=True)
```


In [ ]:
# ModelCheckpointのインポート
from tensorflow.keras.callbacks import ModelCheckpoint

# ModelCheckpointのインスタンスを生成する
checkpoint = ModelCheckpoint(filepath='my_model_Epoch{epoch}_loss{loss:.4f}_valloss{val_loss:.4f}.h5',
                             monitor='val_loss', save_best_only=True, save_weights_only=True)

## 3.モデルの学習と推論の実行

### 3-1.モデルの学習を実行する

```
# Sequentialクラスのインスタンス生成
model = Sequential()
```
**model.fit()メソッドに指定する引数**
```
引数名	データ型	引数の説明
x	NumPy配列	学習に使用する入力データ
y	NumPy配列	学習に使用する正解データ
batch_size	整数	バッチサイズ
epochs	整数	エポック数
validation_data	タプル	検証に使用する入力データと正解データのペア
callbacks	リスト	コールバック
```

In [ ]:
# 検証用データをタプルにまとめる
validation_data = (X_val, Y_val)

# EarlyStoppingの設定
e_stopping = EarlyStopping(monitor='val_loss', patience=30)

# Checkpointの設定
checkpoint = ModelCheckpoint(filepath='my_model_Epoch{epoch}_loss{loss:.4f}_valloss{val_loss:.4f}.h5',
                             monitor='val_loss', save_best_only=True, save_weights_only=True)
                             
# callbacksをリストにまとめる
callbacks = [e_stopping, checkpoint]

# 学習を実行する
model.fit(x=X_train, y=Y_train, batch_size=16, epochs=1,
                 validation_data=validation_data, callbacks=callbacks)

### 3-2.モデルの学習がどのように進行したのかを確認する

**各エポックにおけるMAE評価の推移**
```
グラフから読み取ることのできる情報

1.   MAE評価の推移
学習開始後、しばらくは、loss(学習時のMAE)、val_loss(検証時のMAE)ともに順調に改善を続けている
2.   EarlyStoppingの動作
エポック100あたりでEarlyStoppingが作動し、学習が終了している
3.   未知のデータに対するMAE評価のベストスコア
MAE評価0.085付近で微増減を繰り返していることから、作成したモデルは、MAE評価において誤差約0.085程度の予測精度をもつモデルである
```
**誤差のスケール**
```
衛星画像データを構成する値を255分の1とする正規化処理を行ったことため、誤差のスケールも同じく255分の1となっている
```



### 3-3.学習済みのモデルを使って推論を行う

**1. パラメータが保存されたHDF5ファイルをモデルに読み込む**
```
# パラメータ情報の読み込み
model.load_weights('my_params.h5')
```
**2. 推論の実行**
```
# テスト用データについての推論
Y_pred = model.predict(X_test)
```
**「モデルが生成する予測画像」と「正解画像」の違いについて**
```
*   「台風」の動きを予測できているか
*   「梅雨前線」の動きを予測できているか
*   季節ごとに、どのような予測精度の違いが存在するのか
```

In [ ]:
# 学習済みのパラメータを読み込む
model.load_weights('my_params.h5')

# 推論を実行し、テスト用データの予測値を生成する
Y_pred = model.predict(X_test)

# [5] 予測精度の改善

## 1.特徴量として気象データを追加

### 1-1.気象データを特徴量として加える

```
果たして、衛星画像の未来予測において気象データが特徴量として有効に機能するのかという点に注目し、改めてモデルの学習を実行
```
**気象データを特徴量として追加する方法**

```
チャンネルの次元で連結することで特徴量の追加
np.concatenate()関数

*   第1引数: 連結の対象とする配列、
*   axis: 連結部分となる次元のインデックス

# 衛星画像データと気象データをチャンネルの次元で連結する
X_train = np.concatenate([X_train_sat, X_train_met], axis=4)
```


In [ ]:
X_train = np.concatenate([X_train_sat, X_train_met], axis=4)

print('X_train_satの形状:')
print(X_train_sat.shape)
print('----------------------')
print('X_train_metの形状:')
print(X_train_met.shape)
print('----------------------')
print('X_trainの形状:')
print(X_train.shape)

### 1-2.モデルに変更を加える

```
モデルの入力層の引数input_shapeのチャンネルの次元を衛星画像データのチャンネル数=1から衛星画像データ+気象データのチャンネル数=35に書き換える
```


In [ ]:
model = Sequential()

# Encoder
model.add(ConvLSTM2D(
                    filters=20, kernel_size=(3, 3), padding='same', return_sequences=False,
                    activation='tanh', recurrent_activation='sigmoid',
                    input_shape=(24, 33, 25, 35)))

# EncoderとDecoderをつなぐ中間部分
def repeat_last_status(x):
    x = tf.reshape(x, (-1, 1, 33, 25, 20))
    copied_x = tf.identity(x)
    for _ in range(23):
        x = tf.concat([x, copied_x], axis=1)        
    return x
model.add(Lambda(repeat_last_status))

# Decoder
model.add(ConvLSTM2D(
                    filters=20, kernel_size=(3, 3),
                    padding='same', return_sequences=True,
                    activation='tanh',
                    recurrent_activation='sigmoid'))          

# 出力層
model.add(TimeDistributed(Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')))

### 1-3.気象データを加えた上で再度学習を行った結果を確認する

```
気象データは特徴量として有効に機能する
```